In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
import os

from time import time
from experiment import *

# Load data ** Chose your path before get stated **

In [2]:
path = os.getcwd()
rootDir, _ = path.split('ECG-Arritmia-Paper1')
path = 'ECG-Arritmia-Paper1/Database_extracted/Dataset_with_features'
filename = 'MIT-BIH__DS1_5classes__Fourier.csv'
dataset = pd.read_csv(os.path.join(rootDir, path, filename));

In [3]:
dataset.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,target
0,0.140163,0.302116,0.256512,0.199795,0.167456,0.136428,0.092218,0.053257,0.035375,0.022090,...,0.001537,0.003202,0.000649,0.002842,0.002034,0.003303,0.002120,0.001724,0.001094,0
1,0.164694,0.294092,0.248227,0.189410,0.155726,0.119119,0.084469,0.051921,0.026894,0.022658,...,0.001776,0.001309,0.001864,0.002886,0.001600,0.001463,0.001042,0.000314,0.001563,0
2,0.264069,0.336712,0.279655,0.212017,0.169007,0.126995,0.081884,0.039922,0.025413,0.012168,...,0.002457,0.002108,0.001981,0.001673,0.002745,0.001638,0.001236,0.002879,0.002812,0
3,0.072350,0.351005,0.288275,0.218995,0.177219,0.124068,0.074372,0.044149,0.024169,0.016180,...,0.001813,0.001525,0.002232,0.003359,0.001211,0.002223,0.001470,0.002496,0.000781,0
4,0.029850,0.354420,0.295535,0.229964,0.184878,0.129478,0.076174,0.041396,0.027271,0.015194,...,0.002030,0.000666,0.001806,0.002329,0.001273,0.002032,0.001337,0.002298,0.001719,0


# Data preprocessing

In this step we are going standardize our dataset.

In [4]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()

Separete features from labels.

In [5]:
data_std = std_scaler.fit_transform(dataset.values[:,:-1])
data_label = dataset.values[:,-1]

# Classifiers specifications

In [6]:
# Non-linear models:
from sklearn.neural_network import MLPClassifier

# Kernel-based models
from sklearn.svm import SVC

##### Model pre-loadings:

# MLP:
mlp_clf = MLPClassifier(solver='adam', learning_rate='adaptive', 
                        max_iter=1300, learning_rate_init=5e-04, tol=1e-4)

# SVM
svm_rbf_clf = SVC(kernel='rbf')

## Hyperameter tunning by randomized search:

Classifiers definitions:

In [7]:
classifiers = {'MLP': mlp_clf, 'SVM-RBF': svm_rbf_clf}

Define param range for searching:

In [8]:
param_dist_dict = {'MLP': {"hidden_layer_sizes": list(np.arange(2,1001))},
                   'SVM-RBF': {'gamma': [2*i for i in range(-15,3)], 'C': [2*i for i in range(-5,15)]},
                  }

In [9]:
from sklearn.model_selection import RandomizedSearchCV

random_search = dict((k,[]) for k in classifiers.keys())

for clf in param_dist_dict.keys():
    start = time()
    random_search[clf] = RandomizedSearchCV(classifiers[clf], param_dist_dict[clf], cv=8, n_iter=20, verbose=5, n_jobs=100, scoring='accuracy')
    random_search[clf].fit(data_std, data_label)
    print('Elapsed time:')
    print(time() - start)

Fitting 8 folds for each of 20 candidates, totalling 160 fits
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=863 ..........................................
[CV] hidden_layer_sizes=690 ..........................................
[CV] hidden_layer_sizes=690 ..........................................
[CV] hidden_layer_sizes=690 ..........................................
[CV] hidden_layer_sizes=690 ..........................................
[CV] hidden_layer_sizes=690 ..........................................
[CV] hidden_lay

[Parallel(n_jobs=100)]: Done  27 out of 160 | elapsed: 91.9min remaining: 452.6min


[CV] hidden_layer_sizes=314 ..........................................
[CV] . hidden_layer_sizes=228, score=0.9096045197740112, total=94.6min
[CV] hidden_layer_sizes=314 ..........................................
[CV] . hidden_layer_sizes=350, score=0.8263736263736263, total=94.7min
[CV] hidden_layer_sizes=373 ..........................................
[CV] . hidden_layer_sizes=350, score=0.8889237527455287, total=95.2min
[CV] hidden_layer_sizes=373 ..........................................
[CV] . hidden_layer_sizes=261, score=0.8904926262943207, total=96.4min
[CV] hidden_layer_sizes=373 ..........................................
[CV] . hidden_layer_sizes=261, score=0.9126137433322874, total=96.4min
[CV] hidden_layer_sizes=373 ..........................................
[CV] . hidden_layer_sizes=350, score=0.9013178537809853, total=95.9min
[CV] hidden_layer_sizes=373 ..........................................
[CV] . hidden_layer_sizes=261, score=0.9348610893109403, total=98.5min
[CV] h

[Parallel(n_jobs=100)]: Done  60 out of 160 | elapsed: 143.9min remaining: 239.8min


[CV] hidden_layer_sizes=960 ..........................................
[CV]  hidden_layer_sizes=768, score=0.8568513577146445, total=142.5min
[CV] .. hidden_layer_sizes=683, score=0.82300329515142, total=143.6min
[CV] . hidden_layer_sizes=229, score=0.9049262629432068, total=87.3min
[CV]  hidden_layer_sizes=795, score=0.9058675870724819, total=146.4min
[CV]  hidden_layer_sizes=795, score=0.8571652801757966, total=147.0min
[CV]  hidden_layer_sizes=523, score=0.8322611015220461, total=148.7min
[CV]  hidden_layer_sizes=768, score=0.8793536240978977, total=148.7min
[CV] . hidden_layer_sizes=683, score=0.914783427495292, total=152.5min
[CV] . hidden_layer_sizes=768, score=0.667608286252354, total=152.1min
[CV]  hidden_layer_sizes=646, score=0.8795105114527769, total=156.3min
[CV]  hidden_layer_sizes=768, score=0.9221594475831764, total=157.6min
[CV] . hidden_layer_sizes=229, score=0.8288854003139717, total=92.5min
[CV]  hidden_layer_sizes=863, score=0.8546539004865799, total=165.6min
[CV]  

...

[Parallel(n_jobs=100)]: Done 126 out of 160 | elapsed: 199.8min remaining: 53.9min


[CV] . hidden_layer_sizes=916, score=0.947261026526448, total=201.6min
[CV]  hidden_layer_sizes=768, score=0.9499293674462408, total=201.6min
[CV]  hidden_layer_sizes=314, score=0.8470957613814757, total=108.7min
[CV]  hidden_layer_sizes=672, score=0.9299952911630828, total=122.0min
[CV]  hidden_layer_sizes=373, score=0.8048665620094192, total=104.0min
[CV]  hidden_layer_sizes=916, score=0.8208791208791208, total=206.1min
[CV] . hidden_layer_sizes=314, score=0.865327264165751, total=117.6min
[CV]  hidden_layer_sizes=690, score=0.6001255492780917, total=207.4min
[CV]  hidden_layer_sizes=672, score=0.8304552590266876, total=122.6min
[CV]  hidden_layer_sizes=314, score=0.5364092906465788, total=119.6min
[CV]  hidden_layer_sizes=683, score=0.8703500235441846, total=207.9min
[CV]  hidden_layer_sizes=672, score=0.8364463977397583, total=128.0min
[CV]  hidden_layer_sizes=373, score=0.8455986191746431, total=111.5min
[CV]  hidden_layer_sizes=863, score=0.8046445943825514, total=209.5min
[CV] .

[Parallel(n_jobs=100)]: Done 160 out of 160 | elapsed: 222.0min finished


Elapsed time:
13511.393239021301
Fitting 8 folds for each of 20 candidates, totalling 160 fits
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-6, C=22 ..................................................
[CV] gamma=-2, C=4 ...................................................
[CV] gamma=-2, C=4 ...................................................
[CV] gamma=-2, C=4 ...................................................
[CV] gamma=-2, C=4 ...................................................
[CV] gamma=-2, C=4 ..................................

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/home/navar/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/home/navar/anaconda3/lib/pyth

# Save all models

In [17]:
save_models(random_search)

{'MLP': RandomizedSearchCV(cv=8, error_score='raise',
           estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-08,
        hidden_layer_sizes=(100,), learning_rate='adaptive',
        learning_rate_init=0.0005, max_iter=1300, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=False, warm_start=False),
           fit_params=None, iid=True, n_iter=20, n_jobs=100,
           param_distributions={'hidden_layer_sizes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, ...81, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]},
     

In [12]:
u = random_search['MLP']

In [37]:
u.cv_results_

{'mean_fit_time': array([11203.40463555,  8997.02400357, 11037.82458383,  9367.83068237,
        10884.16404036,  3431.31475157,  5089.2617757 ,  7757.90680751,
        10149.78450951,  5415.78035581,  4592.48680153,  6928.69217107,
         8280.98084357,  5080.64268833,  7283.10500166,  6096.57231277,
         5534.10674149,  4087.35068041,  5082.35533398,  4903.66931403]),
 'std_fit_time': array([1052.05728955, 2437.87878487, 1692.43910445, 2404.22725082,
        1379.11211131,  470.51477868,  792.87852825, 1569.18090969,
        1273.57018397, 1156.69458954,  940.42526566,  967.29892332,
        2042.60045014, 1473.85390409,  345.59162553,  772.82393418,
         770.16944622,  440.48319604,  512.88557497,  259.83303396]),
 'mean_score_time': array([6.90963286, 7.5424706 , 6.3528398 , 7.1949079 , 6.24785355,
        0.9691835 , 2.72083884, 6.87322071, 7.51837429, 3.04232374,
        1.70099995, 4.83447382, 6.6365653 , 2.43057764, 3.37503484,
        2.5384647 , 2.41371399, 1.396902